# Converting PyTorch to TensorFlow Lite for xCORE Using ONNX

ONNX is an open format built to represent machine learning models. We can convert from PyTorch to ONNX, then from ONNX to TensorFlow, then from TensorFlow to TensorFlow Lite, and finally, run it through xformer to optimise it for xCORE.
Ensure that you have installed Python 3.8 and have the installed requirements.txt

## Import PyTorch Model

For this example, we use mobilenet_v2.

In [1]:
import torch

pytorch_model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

# Switch the model to eval mode
pytorch_model.eval()

Using cache found in /Users/salmankhan/.cache/torch/hub/pytorch_vision_v0.10.0


MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

### Run inference on model (to test)

In [2]:
# Download an image to test against
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [3]:
# Download Image Labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2023-03-28 18:20:48--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.18’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.001s  

2023-03-28 18:20:49 (14.0 MB/s) - ‘imagenet_classes.txt.18’ saved [10472/10472]



In [4]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

### Perform an infrence on the pytorch model directly

In [5]:
# We will test and train with these params
batch_size = 1
channels = 3
height = 224
width = 224

In [6]:
from PIL import Image
from torchvision import transforms

# Open testing image
input_image = Image.open(filename)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(height),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Note Pytorch is BCHW
input_tensor = preprocess(input_image)

In [7]:
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

with torch.no_grad():
    output = pytorch_model(input_batch)

probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Show top categories per image
vals, idxs = torch.topk(probabilities, 5)
pytorch_results = [(categories[idx], prob) for (idx, prob) in zip(idxs.tolist(), vals.tolist())]
for cat, prob in  pytorch_results:
    print(cat, ':', prob)

Samoyed : 0.8303043246269226
Pomeranian : 0.06988773494958878
keeshond : 0.012964080087840557
collie : 0.010797776281833649
Great Pyrenees : 0.009886783547699451


## Convert to ONNX


In [8]:
# This is only for shape info for tracnig the model during conversion
sample_input = torch.rand((batch_size, channels, height, width))

onnx_model_path = "mobilenet_v2.onnx"

torch.onnx.export(
    pytorch_model,
    sample_input,
    onnx_model_path,
    input_names=['image'],
    output_names=['probabilities']
)

### Check the exported model

In [9]:
import onnx
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

### Check ONNX Output

In [10]:
import onnxruntime 
import numpy as np
ort_session = onnxruntime.InferenceSession(onnx_model_path)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def softmax(xs):
    return np.exp(xs)/sum(np.exp(xs))

# compute ONNX Runtime output prediction
input_batch_np = to_numpy(input_batch)
ort_inputs = {ort_session.get_inputs()[0].name: input_batch_np}
ort_outs = ort_session.run(None, ort_inputs)

# The input is still BCHW
data = zip(range(len(ort_outs[0][0])), softmax(ort_outs[0][0]))

onnx_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  onnx_results:
    print(cat, ':', prob)

Samoyed : 0.830305
Pomeranian : 0.06988746
keeshond : 0.012964004
collie : 0.010797733
Great Pyrenees : 0.009886744


## Using onnx2tf

Using package: https://github.com/PINTO0309/onnx2tf

### Convert ONNX to Keras

In [11]:
import onnx2tf

keras_model_path = 'mobilenet_v2.tf'

keras_model = onnx2tf.convert(
    input_onnx_file_path=onnx_model_path,
    output_folder_path=keras_model_path,
    non_verbose=True,
)

### Check the conversion to keras

In [12]:
#transpose the input_batch into BHWC order for tensorflow
tf_input_data = np.transpose( input_batch.numpy(), [0, 2, 3, 1])

keras_output_data = keras_model(tf_input_data)

probs = softmax(keras_output_data[0])
data = zip(range(len(probs)), probs)
keras_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  keras_results:
    print(cat, ':', prob)

Samoyed : 0.83030456
Pomeranian : 0.069887355
keeshond : 0.01296406
collie : 0.01079778
Great Pyrenees : 0.009886805


## Convert Keras to TFLite (float32)

In [13]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter.inference_input_type = tf.float32 
converter.inference_output_type = tf.float32

tflite_model = converter.convert()

# Save the model.
tflite_float_model_path = 'mobilenet_v2_float.tflite'
with open(tflite_float_model_path, 'wb') as f:
  f.write(tflite_model)

### Check it Worked

In [14]:
tfl_interpreter = tf.lite.Interpreter(model_path=tflite_float_model_path)
tfl_interpreter.allocate_tensors()

tfl_input_details = tfl_interpreter.get_input_details()
tfl_output_details = tfl_interpreter.get_output_details()

# Convert PyTorch Input Tensor into Numpy Matrix and Reshape for TensorFlow
# (Pytorch model expects C x H x W but TF expects H x W x C)
tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tf_input_data)
tfl_interpreter.invoke()

tfl_output_data = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

probs = softmax(tfl_output_data[0])
data = zip(range(len(probs)), probs)
tfl_float32_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  tfl_float32_results:
    print(cat, ':', prob)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Samoyed : 0.84704113
Pomeranian : 0.060772426
keeshond : 0.012674405
collie : 0.00844481
Great Pyrenees : 0.008021476


## Convert Keras to TFLite (int8)
We will still feed the data into the model in float32 format for convinence but the internals of the model will be int8. This will require representitive data but as we interface in float32 we can use the pytorch preprocessing. 

## Representative Dataset

To convert a model into to a TFLite flatbuffer, a representative dataset is required to help in quantisation. Refer to [Converting a keras model into an xcore optimised tflite model](https://colab.research.google.com/github/xmos/ai_tools/blob/develop/docs/notebooks/keras_to_xcore.ipynb) for more details on this.

#### Download & Extract Images from Data Set

In [15]:
# We will use the imagenette dataset(326MB)
!mkdir ./imagenet-dataset
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz -O ./imagenet-dataset/imagenette2-320.tgz
!tar -xf ./imagenet-dataset/imagenette2-320.tgz -C ./imagenet-dataset/

mkdir: ./imagenet-dataset: File exists
--2023-03-28 18:21:46--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.171.5, 52.217.111.62, 52.217.225.136, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.171.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 341663724 (326M) [application/x-tar]
Saving to: ‘./imagenet-dataset/imagenette2-320.tgz’

./imagenet-dataset/ 100%[===================>] 325.83M  7.95MB/s    in 47s     

2023-03-28 18:22:34 (6.90 MB/s) - ‘./imagenet-dataset/imagenette2-320.tgz’ saved [341663724/341663724]



In [16]:
import os
import glob
import random

all_files = glob.glob(os.path.join("./imagenet-dataset/", '**/*.JPEG'), recursive=True)

# Randomly select a subset of images, let's just use 1k for speed
sampled_files = random.sample(all_files, 1000)

def representative_dataset():

    # Iterate over the sampled images and preprocess them
    for image_path in sampled_files:
        pil_image = Image.open(image_path).convert("RGB")
        pytorch_tensor = preprocess(pil_image).unsqueeze(0)
        img_array_np = pytorch_tensor.numpy()
        
        #swap the axes to BHWC(tf) from BCHW(pytorch)
        img_array_np = np.transpose( img_array_np, [0, 2, 3, 1])
        yield [img_array_np]

In [17]:
# Now do the conversion to int8
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32 
converter.inference_output_type = tf.float32

tflite_int8_model = converter.convert()

# Save the model.
tflite_int8_model_path = 'mobilenet_v2.tflite'
with open(tflite_int8_model_path, 'wb') as f:
  f.write(tflite_int8_model)

fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32


In [18]:
tfl_interpreter = tf.lite.Interpreter(model_path=tflite_int8_model_path)
tfl_interpreter.allocate_tensors()

tfl_input_details = tfl_interpreter.get_input_details()
tfl_output_details = tfl_interpreter.get_output_details()

# Convert PyTorch Input Tensor into Numpy Matrix and Reshape for TensorFlow
tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tf_input_data)
tfl_interpreter.invoke()

tfl_output_data = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

probs = softmax(tfl_output_data[0])
data = zip(range(len(probs)), probs)
tfl_int8_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  tfl_int8_results:
    print(cat, ':', prob)

Samoyed : 0.8527114
Pomeranian : 0.075921215
keeshond : 0.018518537
Arctic fox : 0.0067596436
West Highland white terrier : 0.0055256966


# Analysing Models

Defined below is a function to print out the operator counts of a model.

In [19]:
import io
from contextlib import redirect_stdout

def get_operator_counts(model_content):
    with io.StringIO() as buf, redirect_stdout(buf):
        tf.lite.experimental.Analyzer.analyze(model_content=model_content)
        model_structure = buf.getvalue()

    operators = [op.strip().split(" ")[1].split("(")[0] for op in model_structure.split("\n") if "Op#" in op]
    op_counts = {}
    for operator in operators:
        if operator in op_counts:
            op_counts[operator] = op_counts[operator]+1
        else:
            op_counts[operator] = 1
        
    return (len(operators), op_counts)

def print_operator_counts(model_content):
    total_op_count, op_counts = get_operator_counts(model_content)
    print(f"{'Operator'.upper():<20} {'Count'.upper():>6}")
    print("-"*20 + " " + "-"*6)
    
    for operator, count in op_counts.items():
        print(f"{operator.lower():<20} {count:>6}")
        
    print("-"*20 + " " + "-"*6)
    print(f"{'Total'.upper():<20} {total_op_count:>6}")
    print("-"*20 + " " + "-"*6)

In [20]:
# Let's inspect the int8 model
print_operator_counts(tflite_int8_model)

OPERATOR              COUNT
-------------------- ------
quantize                  1
pad                       5
conv_2d                  35
depthwise_conv_2d        17
add                      10
mean                      1
fully_connected           1
dequantize                1
-------------------- ------
TOTAL                    71
-------------------- ------


## Compare Accuracy

In [21]:
import tensorflow_datasets as tfds
import tensorflow as tf
import requests

# download labels
resp = requests.get('https://raw.githubusercontent.com/tensorflow/models/master/research/slim/datasets/imagenet_metadata.txt')
# make a dictionary of classes, e.g: {"n00440382": ["broad", "jump", "long jump"}
labels_dict = {x[0]: [y.strip() for y in x[1].split(",")] for x in [x.split("\t") for x in resp.text.splitlines()]}


# Construct a tf.data.Dataset
ds, info = tfds.load('imagenette', split='validation', with_info=True, as_supervised=True, shuffle_files=True)
label_codes = info.features['label']

# make lowercase to standardise
torch_labels = [x.lower() for x in categories]

def is_label_in_both_datasets(labels):
    return True in [imagenet_label.lower() in torch_labels for imagenet_label in labels]

def is_correct_result(result, expected):
    return result in expected


# Build your input pipeline
ds = ds.shuffle(2000).batch(1).prefetch(10).take(2000)

iterations = 0
correct = 0

for image, label in ds:
    # get actual words for the image classification
    label_terms = labels_dict[label_codes.int2str(label.numpy()[0])]
    
    # only compare images where the label exists in both the dataset that the model was originally trained on, and in the tfds dataset
    if is_label_in_both_datasets(label_terms):
        iterations = iterations + 1
        
        img = tf.keras.utils.array_to_img(image[0])
        pytorch_batch = preprocess(img).unsqueeze(0)
        tf_batch = np.transpose( pytorch_batch.numpy(), [0, 2, 3, 1])

        # use same interpreter as before
        tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tf_batch)
        tfl_interpreter.invoke()

        output = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

        probs = softmax(output[0])
        data = zip(range(len(probs)), probs)
        (idx, prob) = sorted(data, key=lambda x: x[1], reverse=True)[0]
        result = torch_labels[idx]
        
        if is_correct_result(result, label_terms):
            correct = correct + 1

accuracy = correct / iterations
print(f"Accuracy {correct}/{iterations} = {accuracy * 100}%")

Accuracy 1303/2000 = 65.14999999999999%
